In [1]:
# 내 구글 드라이브에 연동
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# 필요한 패키지와 모듈을 불러옴
import cv2
import numpy as np
import time
import io
import base64
from IPython.display import HTML

In [3]:
# Detection 하기 전에 원본 동영상을 Display
video = io.open('/content/gdrive/MyDrive/Helmet_Detection/img/media.mp4', 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video width="50%" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4"/>
             </video>'''.format(encoded.decode('ascii')))

Output hidden; open in https://colab.research.google.com to view.

In [4]:
file_name = '/content/gdrive/MyDrive/Helmet_Detection/img/media.mp4'  # Detection 할 원본 동영상
min_confidence = 0.2   # detection 으로 인정할 최소 확률(신뢰도) 지정
nms_threshold = 0.4    # non-max suppression threshold
output_name = 'output_media.mp4'  # Detection 된 output 동영상 
elapsed_time = 0       # 총 경과시간 초기화 

In [5]:
def detectAndDisplay(frame):
    start_time = time.time()
    img = cv2.resize(frame, None, fx=0.9, fy=0.9)
    height, width, channels = img.shape

    # YOLOv3의 Detecting model 3가지(320×320, 416×416, 608×608)
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), swapRB=True, crop=False)

    net.setInput(blob)
    outs = net.forward(output_layers)

    # Showing informations on the screen
    class_ids = []      # detection 한 Class id를 저장하는 배열 정의
    confidences = []    # detection 한 Class 의 신뢰도(확률)를 저장하는 배열 정의
    boxes = []          # detection 한 boxing 정보를 저장하는 배열 정의
    colors = []        # detection 한 Class id 마다 다른 컬러를 지정하는 배열 정의

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)      # detection 한 Class id
            confidence = scores[class_id]     # detection 한 Class 의 신뢰도(확률)
            if confidence > min_confidence:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])             # boxing 정보를 boxes 배열에 저장
                confidences.append(float(confidence))  # 신뢰도(확률)을 confidences 배열에 저장
                class_ids.append(class_id)             # Class id 를 class_ids 배열에 저장
                colors.append(color_lists[class_id])   # Class id 마다 다른 컬러를 배열에 저장
    
    # apply non-max suppression
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, min_confidence, nms_threshold)    # 박스안에 박스(노이즈)를 하나로 만들어 준다.
    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:    # 노이즈가 제거된 박스만 표시해 준다.   
            x, y, w, h = boxes[i]
            label = "{}: {:.2f}".format(classes[class_ids[i]], confidences[i]*100)    # Class 이름, 신뢰도(확률) 표시 
            print(i, label)
            color = colors[i]
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 1)
            cv2.rectangle(img, (x, y - 25), (x + 125, y), color, -1)
            cv2.putText(img, label, (x + 2, y - 10), font, 1, (255, 255, 255), 1)
            
    process_time = time.time() - start_time
    global elapsed_time
    elapsed_time += process_time   # 총 경과시간 누적
    print("=== A frame took {:.3f} seconds".format(process_time))

    # video 를 disk 에 output 하기 위해 writer 를 초기화한다.
    global writer
    if writer is None and output_name is not None:
        fourcc = cv2.VideoWriter_fourcc(*"DIVX")
        writer = cv2.VideoWriter(output_name, fourcc, 20,
                (img.shape[1], img.shape[0]), True)
        
    # disk 에 frame 을 write 합니다.
    if writer is not None:
        writer.write(img)

In [6]:
# Load YOLO
net = cv2.dnn.readNet("/content/gdrive/MyDrive/Helmet_Detection/backup/custom-train-yolo3_last.weights", "/content/gdrive/MyDrive/Helmet_Detection/custom/custom-test-yolo3.cfg")
classes = []   # detection 할 Object(Class) list 배열을 정의
with open("/content/gdrive/MyDrive/Helmet_Detection/custom/classes.names", "r") as f:
   classes = [line.strip() for line in f.readlines()]   # 3개의 Object(class)를 구분할 수 있는 Object의 이름을 classes 배열에 넣어준다.
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
color_lists = np.random.uniform(0, 255, size=(len(classes), 3))  # Object 마다 컬러를 하나씩 다르게 지정

# 원본 동영상에서 video stream을 읽어온다.
cap = cv2.VideoCapture(file_name)
writer = None
if not cap.isOpened:
    print('--(!)Error opening video capture')
    exit(0)
while True:
    ret, frame = cap.read()
    if frame is None:
        # close the video file pointers
        cap.release()
        # close the writer point
        writer.release()
        print('--(!) No captured frame -- Break!')
        print("elapsed time {:.3f} seconds".format(elapsed_time))
        break
    detectAndDisplay(frame)

0 helmet: 99.37
=== A frame took 1.817 seconds
0 helmet: 99.32
=== A frame took 1.086 seconds
0 helmet: 98.94
=== A frame took 1.068 seconds
0 helmet: 99.09
=== A frame took 1.068 seconds
0 helmet: 99.39
=== A frame took 1.086 seconds
0 helmet: 99.39
=== A frame took 1.093 seconds
0 helmet: 99.33
=== A frame took 1.066 seconds
0 helmet: 99.36
=== A frame took 1.124 seconds
0 helmet: 99.42
=== A frame took 1.110 seconds
0 helmet: 99.44
=== A frame took 1.115 seconds
0 helmet: 99.45
=== A frame took 1.089 seconds
0 helmet: 99.45
=== A frame took 1.071 seconds
0 helmet: 99.51
=== A frame took 1.082 seconds
0 helmet: 99.48
=== A frame took 1.082 seconds
0 helmet: 99.44
=== A frame took 1.072 seconds
0 helmet: 99.38
=== A frame took 1.108 seconds
0 helmet: 99.39
=== A frame took 1.076 seconds
0 helmet: 99.42
=== A frame took 1.062 seconds
0 helmet: 99.51
=== A frame took 1.067 seconds
0 helmet: 99.43
=== A frame took 1.066 seconds
0 helmet: 99.46
=== A frame took 1.065 seconds
0 helmet: 99.